In [1]:
!pip install pydub

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
from pydub import AudioSegment
from google.colab import drive

# drive.flush_and_unmount()
drive.mount('/content/drive')

input_path = '/content/drive/MyDrive/dataset/MPaudio1.mp4'
output_path = '/content/extracted_audio.wav'

ffmpeg_extract_audio(input_path, output_path)
audio = AudioSegment.from_file(output_path)

duration = 3 * 60 * 1000
trimmed_audio = audio[:duration] #for first 3 minutes

# trimmed_audio = audio # for final data
trimmed_audio.export('/content/trimmed_audio.wav', format='wav')

# Stereo to Mono
stereo_audio = AudioSegment.from_file('/content/trimmed_audio.wav')
mono_audio = stereo_audio.set_channels(1)
mono_audio.export('/content/final_audio.wav',format='wav')

output_path = '/content/final_audio.wav'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Moviepy - Running:
>>> "+ " ".join(cmd)


In [ ]:
num_speakers = 2
language = 'English'
model_size = 'large'


In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
path = '/content/final_audio.wav'

In [ ]:
model = whisper.load_model(model_size)

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("/content/transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:
file_path = '/content/transcript.txt'

with open(file_path, "r") as file:
  results = file.read()

print(results)

In [ ]:
import os
file_path = "/content/final_audio.wav"
file_size = os.path.getsize(file_path)

file_size_mb = file_size / (1024 * 1024)
print("File size:", "{:.2f} MB".format(file_size_mb))

import shutil

source_path = '/content/final_audio.wav'
destination_path = '/content/drive/MyDrive/final_audio.wav'
shutil.copyfile(source_path, destination_path)